In [1]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [2]:
!kaggle datasets download -d salader/dogs-vs-cats


Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
100% 1.06G/1.06G [00:11<00:00, 83.5MB/s]
100% 1.06G/1.06G [00:11<00:00, 98.6MB/s]


In [3]:
import zipfile
import os

# Unzip the dataset
with zipfile.ZipFile("dogs-vs-cats.zip", 'r') as zip_ref:
    zip_ref.extractall("dogs-vs-cats")

# Verify the extraction
print("Extracted files:", os.listdir("dogs-vs-cats"))


Extracted files: ['train', 'test', 'dogs_vs_cats']


In [4]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten


In [5]:
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/dogs-vs-cats/train',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (256, 256)
)
validation_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/dogs-vs-cats/test',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (256, 256)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [6]:
#normalise
def process(image, label):
  image = tf.cast(image/255. ,tf.float32)
  return image, label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [7]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), padding='valid', activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Conv2D(64, kernel_size=(3,3), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Conv2D(128, kernel_size=(3,3), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
history = model.fit(train_ds , epochs = 10 , validation_data = validation_ds)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 58s 78ms/step - accuracy: 0.5948 - loss: 0.6809 - val_accuracy: 0.7490 - val_loss: 0.5161
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 69s 70ms/step - accuracy: 0.7521 - loss: 0.5113 - val_accuracy: 0.7870 - val_loss: 0.4615
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 84s 73ms/step - accuracy: 0.8207 - loss: 0.3922 - val_accuracy: 0.7830 - val_loss: 0.5891
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 81s 71ms/step - accuracy: 0.8937 - loss: 0.2532 - val_accuracy: 0.7786 - val_loss: 0.7225
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 46s 73ms/step - accuracy: 0.9500 - loss: 0.1418 - val_accuracy: 0.7872 - val_loss: 0.8695
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 81s 72ms/step - accuracy: 0.9670 - loss: 0.0920 - val_accuracy: 0.7806 - val_loss: 0.9944
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 46s 73ms/step - accuracy: 0.9796 - loss: 0.0581 - val_accuracy: 0.7628 - val_loss: 1.1929
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 45s 71ms/step - accuracy: 0.9821 - loss: 0.0506 - 